In [130]:
import numpy as np
import random
import math

In [131]:
EMPTY = 0
CROSSES = 1
CIRCLES = 2

ROWS = 3
COLUMNS = 3
DIAGONALS = 2

WIN = 1
DRAW = 0.5
LOSS = 0

In [132]:
start_board = np.array((EMPTY, EMPTY, EMPTY, EMPTY, CROSSES, EMPTY, EMPTY, EMPTY, EMPTY))
converter = lambda _: {EMPTY: " ", CROSSES: "X", CIRCLES: "O"}[_]

In [133]:
game_moves = [("X", 4)]

In [134]:
def possible_move(board: np.array) -> list:
    return [i for i in range(len(board)) if board[i] == EMPTY]

In [135]:
def opponent_move(board: np.array) -> np.array:
    possible_moves = possible_move(board)
    if len(possible_moves) != 0:
        move = random.choice(possible_moves)
        game_moves.append(("X", move))
        board[move] = CROSSES
        return board
    else:
        raise KeyError("Board is full")

In [136]:
def print_board( board: np.array) -> None:
    data = np.reshape(board, (ROWS, COLUMNS))

    for i, row in enumerate(data):
        print(
            f" {converter(row[0])} | {converter(row[1])} | {converter(row[2])}"
        )
        if i != len(data) - 1:
            print("---+---+---")

In [137]:
def winning(board: np.array) -> tuple[bool, int]:
    # ROWS
    for i in range(0, ROWS*DIAGONALS, ROWS):
        if board[i] == board[i + 1] == board[i + 2] and board[i] != EMPTY:
            return True, board[i]

    # COLUMNS
    for j in range(COLUMNS):
        if board[j] == board[j + 3] == board[j + 6] and board[j] != EMPTY:
            return True, board[j]

    # DIAGONALS (start top left to bottom right, then top right to bottom left)
    for k in range(DIAGONALS):
        if board[0 if k == 0 else 2] == board[4] == board[8 if k == 0 else 6]:
            return True, board[k]

    # No winner yet
    return False, None

In [138]:
def selection(board: np.array, node: list = []) -> tuple[list, np.array]:
    if random.choice([True, False]):
        return node, board
    else:
        move = random.choice(possible_move(board))
        board[move] = CIRCLES
        node.append(move)
        game_moves.append(("O", move))
        # Stop if this is winning terminating node
        if winning(board)[0]: return node, board
        board = opponent_move(board)
        # Stop if this is losing terminating node
        if winning(board)[0]: return node, board
        return selection(board, node)

In [139]:
game_moves = [("X", 4)]
selection_moves, selection_board = selection(start_board.copy())
print_board(selection_board)
print(selection_moves)
print(game_moves)

   |   |  
---+---+---
   | X |  
---+---+---
   |   |  
[]
[('X', 4)]


In [140]:
def expansion(board: np.array) -> list:
    children = []
    for move in possible_move(board):
        # Copy data
        child_board = board.copy()
        child_moves = game_moves.copy()

        # Adjust for child
        child_board[move] = CIRCLES
        child_moves.append(("O", move))
        children.append((child_moves, child_board))

    return children

In [141]:
for m, b in expansion(selection_board):
    print_board(b)
    print(m)

 O |   |  
---+---+---
   | X |  
---+---+---
   |   |  
[('X', 4), ('O', 0)]
   | O |  
---+---+---
   | X |  
---+---+---
   |   |  
[('X', 4), ('O', 1)]
   |   | O
---+---+---
   | X |  
---+---+---
   |   |  
[('X', 4), ('O', 2)]
   |   |  
---+---+---
 O | X |  
---+---+---
   |   |  
[('X', 4), ('O', 3)]
   |   |  
---+---+---
   | X | O
---+---+---
   |   |  
[('X', 4), ('O', 5)]
   |   |  
---+---+---
   | X |  
---+---+---
 O |   |  
[('X', 4), ('O', 6)]
   |   |  
---+---+---
   | X |  
---+---+---
   | O |  
[('X', 4), ('O', 7)]
   |   |  
---+---+---
   | X |  
---+---+---
   |   | O
[('X', 4), ('O', 8)]


In [149]:
# Play random game
def rollout(moves: list, board: np.array) -> tuple[list, np.array, float]:
    rollout_board = board.copy()
    game = moves
    player = CROSSES
    score = None
    while True:
        pos_moves = possible_move(rollout_board)
        move = random.choice(pos_moves)
        rollout_board[move] = player
        game.append((converter(player), move))
        player =  CIRCLES if player == CROSSES else CROSSES #flip player

        # ending conditions (winning or draw)
        t, w = winning(rollout_board)
        if t: 
            score = WIN if w == CIRCLES else LOSS
            break

        if len(pos_moves) == 1:
            score = DRAW
            break
    
    return game, rollout_board, score

In [143]:
# Try all possible solutions
# def rollout(board: np.array) -> tuple[float, int]:
#     score = 0
#     #first opponent move
#     for opm in possible_move(board):
#         # Update board
#         new_board = board.copy()
#         new_board[opm] = CROSSES

#         t, w = winning(new_board)
#         if t: 
#             score += 1 if w == CIRCLES else 0
#         elif len(possible_move(new_board)) == 0:
#             score += 0.5
            
#         else:
#             # our move
#             for usm in possible_move(new_board):
#                 # Update board
#                 newer_board = new_board.copy()
#                 newer_board[usm] = CIRCLES

#                 t, w = winning(newer_board)
#                 if t: 
#                     score += 1 if w == CIRCLES else 0
#                 elif len(possible_move(newer_board)) == 0:
#                     score += 0.5
#                 else:
#                     score += rollout(new_board)[0] 

#     n = math.factorial(len(possible_move(board)))
#     return score, n

In [150]:
for m, b in expansion(selection_board):
    #print_board(b)
    # print(m)

    game, _board, score = rollout(m, b)
    # print(score)
    # print(n)
    print(game)
    print_board(_board)
    print(score)
    # print(f"win percentage: {score}")

[('X', 4), ('O', 0), ('X', 2), ('O', 7), ('X', 6)]
 O |   | X
---+---+---
   | X |  
---+---+---
 X | O |  
0
[('X', 4), ('O', 1), ('X', 6), ('O', 7), ('X', 5), ('O', 2), ('X', 8), ('O', 3), ('X', 0)]
 X | O | O
---+---+---
 O | X | X
---+---+---
 X | O | X
0
[('X', 4), ('O', 2), ('X', 3), ('O', 0), ('X', 7), ('O', 8), ('X', 5)]
 O |   | O
---+---+---
 X | X | X
---+---+---
   | X | O
0
[('X', 4), ('O', 3), ('X', 0), ('O', 7), ('X', 6), ('O', 8), ('X', 2)]
 X |   | X
---+---+---
 O | X |  
---+---+---
 X | O | O
0
[('X', 4), ('O', 5), ('X', 7), ('O', 0), ('X', 3), ('O', 6), ('X', 8), ('O', 2), ('X', 1)]
 O | X | O
---+---+---
 X | X | O
---+---+---
 O | X | X
0
[('X', 4), ('O', 6), ('X', 7), ('O', 2), ('X', 1)]
   | X | O
---+---+---
   | X |  
---+---+---
 O | X |  
0
[('X', 4), ('O', 7), ('X', 5), ('O', 3), ('X', 6), ('O', 2), ('X', 1), ('O', 8), ('X', 0)]
 X | X | O
---+---+---
 O | X | X
---+---+---
 X | O | O
0.5
[('X', 4), ('O', 8), ('X', 6), ('O', 0), ('X', 3), ('O', 7), ('X', 1